In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [3]:
station = 0
overlap = 0
result = []

In [4]:
rootdir = '/home/cz2397/data/cchdo-hyd/'

In [5]:
client = MongoClient()
db = client.eddies
rcs_cchdo = db.rcs_cchdo

In [6]:
rcs_cchdo.create_index([('dat', pymongo.ASCENDING)])

u'dat_1'

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('hy1.nc'):
            path = os.path.join(subdir, file)
            DS = xr.open_dataset(path)
            DAT = DS.time.values[0]
            LAT = float(DS.latitude.values[0])
            LON = float(format(DS.longitude.values[0]))
            dat_var = np.timedelta64(4, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$center': [[LON, LAT], 1]}}
            }
            count = rcs_cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(path)

100%|██████████| 152/152 [00:05<00:00, 26.88it/s]


In [8]:
station

39790

In [9]:
overlap

838

In [10]:
result.sort()

In [11]:
result

['/home/cz2397/data/cchdo-hyd/114_32MW066_1_nc_hyd/hot066_00001_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00002_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00003_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00004_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00005_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00006_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00007_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00008_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00009_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00010_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00011_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/117_318M171_1_nc_hyd/PRS2_00002_00012_hy1.nc